In [1]:
import math
from sympy import symbols, sqrt, diff, Symbol, latex

This notebook uses `sympy` to perform symbolic differentiation to help with writing the manual backward pass for each order. To maintain consistency, the forward pass equations are direct copies/transcribed from the `e3nn` spherical harmonics functions, and for that reason we break up the sections in this notebook that "match" the `e3nn` implementation.

In [2]:
x, y, z = symbols("x y z")

In [3]:
def take_derivative(expr, symbols: list[Symbol], simplify: bool = True):
    """
    Function to take the derivative of a symbolic equation with respect
    to a list of symbols.

    We loop through each symbol, and if it is used in the equation,
    we take the first derivative with respect to that function.
    """
    return_dict = {}
    for symbol in symbols:
        if symbol in expr.free_symbols:
            deriv = diff(expr, symbol)
            if simplify:
                deriv = deriv.simplify()
            return_dict[str(symbol)] = deriv
    if len(return_dict) == 0:
        raise RuntimeError("None of the requested symbols were used in the expression!")
    return return_dict

In [4]:
def collect_derivatives(derivs: dict, l: int, simplify: bool = True) -> dict:
    """Collect up derivatives from each component in terms of a cartesian axis"""
    joint = {}
    for axis in ["x", "y", "z"]:
        for m, components in derivs.items():
            # we use a dYl^m symbol to denote that you need to multiply
            # by this particular component's gradient
            m_symbol = symbols(f"dY{l}^{m}")
            # not every component contributes gradients to an axis
            if axis in components:
                if axis not in joint:
                    joint[axis] = components[axis] * m_symbol
                else:
                    joint[axis] += components[axis] * m_symbol
    if simplify:
        for axis, expr in joint.items():
            joint[axis] = expr.simplify()
    return joint

In [5]:
"""
This cell implements helper functions that will reformat the sympy/Python string result
into something that can be more readily copy-pasted for the Triton implementation.
"""


def replace_terms_for_implementation(e: str, mapping: dict[str, str]) -> str:
    """
    Implements a function that remaps a sympy expression to match
    the syntax used in the actual Triton implementation
    """
    for key, value in mapping.items():
        e = e.replace(key, value)
    return e


mapping = {
    "sqrt(15)": "sqrt_15",
    "x**4.0": "sq_x * sq_x",
    "z**4.0": "sq_z * sq_z",
    "x**4": "sq_x * sq_x",
    "z**4": "sq_z * sq_z",
}

for power, prefix in zip([1, 2, 3], ["", "sq_", "cu_"]):
    for axis in ["x", "y", "z"]:
        # this one comes the decimal one comes first, otherwise
        # it leaves dangling .0
        mapping[f"{axis}**{power}.0"] = f"{prefix}{axis}"
        mapping[f"{axis}**{power}"] = f"{prefix}{axis}"

"""
This adds the gradient terms to the re-mapping dictionary; ideally
this makes it so that everything is literally just copy paste!
"""


def num_projections(l: int) -> int:
    return 2 * l + 1


for l_max in range(1, 4 + 1):
    for m in range(num_projections(l_max)):
        mapping[f"dY{l_max}^{m}"] = f"g_{l_max}_{m}"


def export_to_latex(expr) -> None:
    """Function to format a sympy expression for copy/pasting into LaTeX"""
    expr_str = latex(expr)
    expr_mapping = {
        "dY": "\\nabla Y",
        "^{1.0}": "",
        "^{2.0}": "^2",
        "^{3.0}": "^3",
        "^{4.0}": "^4",
    }
    for key, value in expr_mapping.items():
        expr_str = expr_str.replace(key, value)
    print(expr_str)

## First order spherical harmonics

...is just constant because it's just $\sqrt{3}(x,y,z)$

In [6]:
zeroth_x = x * sqrt(3.0)

In [7]:
diff(zeroth_x, x)

1.73205080756888

## Second order spherical harmonics

A little bit more involved; note I'm butchering the syntax where it's normally meant to be $y_l^m$, but I'm using $m$ indexing as if it was written in `e3nn` code.

$$ y_2^0 = \sqrt{15}xz $$

$$ y_2^1 = \sqrt{15}xy $$

$$ y_2^3 = \sqrt{15}yz $$

$$ y_2^4 = \sqrt{5} (y^2 - \frac{1}{2}(x^2 + z^2) $$

$$ y_2^5 = \frac{\sqrt{15}}{2} (z^2 - x^2) $$

In [8]:
y_2_0 = sqrt(15) * x * z
y_2_1 = sqrt(15) * x * y
y_2_2 = sqrt(15) * y * z
y_2_3 = sqrt(5) * (y**2 - 0.5 * (x**2 + z**2))
y_2_4 = (sqrt(15) / 2) * (z**2.0 - x**2.0)

In [9]:
second_order = {}
for index, expr in enumerate([y_2_0, y_2_1, y_2_2, y_2_3, y_2_4]):
    second_order[str(index)] = take_derivative(expr, [x, y, z])

The cell below shows how each projection of $l$ contributes to $x,y,z$ axes. It's a convenient way to inspect the appearance of terms, but not so much for the actual implementation.

In [10]:
second_order

{'0': {'x': sqrt(15)*z, 'z': sqrt(15)*x},
 '1': {'x': sqrt(15)*y, 'y': sqrt(15)*x},
 '2': {'y': sqrt(15)*z, 'z': sqrt(15)*y},
 '3': {'x': -1.0*sqrt(5)*x, 'y': 2*sqrt(5)*y, 'z': -1.0*sqrt(5)*z},
 '4': {'x': -1.0*sqrt(15)*x**1.0, 'z': 1.0*sqrt(15)*z**1.0}}

Instead, we call `collect_derivatives` to aggregate each expression, and reorder it so that it's in terms of the derivatives of _each projection_ with respect to $x,y,z$. This makes it so that the Triton implementation is straightforward, since it maps onto how backprop actually produces gradients.

In [11]:
second_order_cartesian = collect_derivatives(second_order, 2)

So the cell below derives $\frac{\partial Y_2}{\partial x}$; i.e. how each projection of 2nd order spherical harmonic contributes to the total gradient of $x$.

In [12]:
replace_terms_for_implementation(str(second_order_cartesian["x"]), mapping)

'1.0*sqrt_15*g_2_0*z + 1.0*sqrt_15*g_2_1*y - 1.0*sqrt(5)*g_2_3*x - 1.0*sqrt_15*g_2_4*x'

In [13]:
replace_terms_for_implementation(str(second_order_cartesian["y"]), mapping)

'sqrt_15*g_2_1*x + sqrt_15*g_2_2*z + 2*sqrt(5)*g_2_3*y'

In [14]:
replace_terms_for_implementation(str(second_order_cartesian["z"]), mapping)

'1.0*sqrt_15*g_2_0*x + 1.0*sqrt_15*g_2_2*y - 1.0*sqrt(5)*g_2_3*z + 1.0*sqrt_15*g_2_4*z'

The remainder of this notebook follows the exact same recipe, and will not be documented as thoroughly as it is mainly for the implementation/reference. Some cells include a `print(latex(...))` line, which is used to pretty-print/format for $\LaTeX$ outputs.

## Third order spherical harmonics

$$ y_3^0 = \frac{1}{6} \sqrt{42} ({y_2^0} z + {y_2^4} x) $$

$$ y_3^1 = \sqrt{7} y_2^0 y $$

$$ y_3^2 = \frac{1}{8} \sqrt{168} (4y^2 - x^2 + z^2) x $$

$$ y_3^3 = \frac{1}{2} \sqrt{7} y (2y^2 - 3(x^2 + z^2)) $$

$$ y_3^4 = \frac{1}{8} \sqrt{168} z(4y^2 - (x^2 + z^2)) $$

$$ y_3^5 = \sqrt{7} y_2^4 y $$

$$ y_3^6 = \frac{1}{6} \sqrt{42} (y_2^4 z - y_2^0 x) $$

In [15]:
y2 = y**2.0
x2z2 = x**2.0 + z**2.0

y_3_0 = (1 / 6) * math.sqrt(42) * (y_2_0 * z + y_2_4 * x)
y_3_1 = math.sqrt(7) * y_2_0 * y
y_3_2 = (1 / 8) * math.sqrt(168) * (4.0 * y2 - x2z2) * x
y_3_3 = (1 / 2) * math.sqrt(7) * y * (2.0 * y2 - 3.0 * x2z2)
y_3_4 = (1 / 8) * math.sqrt(168) * z * (4.0 * y2 - x2z2)
y_3_5 = math.sqrt(7) * y_2_4 * y
y_3_6 = (1 / 6) * math.sqrt(42) * (y_2_4 * z - y_2_0 * x)

In [16]:
third_order = {}
for index, expr in enumerate([y_3_0, y_3_1, y_3_2, y_3_3, y_3_4, y_3_5, y_3_6]):
    third_order[str(index)] = take_derivative(expr, [x, y, z])

In [17]:
third_order_cartesian = collect_derivatives(third_order, 3)

In [18]:
export_to_latex(third_order_cartesian["x"])

\sqrt{15} \nabla Y^{0}_{3} \left(- 1.62018517460196 x^2 + 1.08012344973464 z^{2} + 0.540061724867322 z^2\right) + 2.64575131106459 \sqrt{15} \nabla Y^{1}_{3} y z - \nabla Y^{2}_{3} \cdot \left(4.8605555238059 x^2 - 6.48074069840786 y^2 + 1.62018517460197 z^2\right) - 7.93725393319377 \nabla Y^{3}_{3} x y - 3.24037034920393 \nabla Y^{4}_{3} x z - 2.64575131106459 \sqrt{15} \nabla Y^{5}_{3} x y - \sqrt{15} \nabla Y^{6}_{3} z \left(1.08012344973464 x + 2.16024689946929 x\right)


In [19]:
replace_terms_for_implementation(str(third_order_cartesian["x"]), mapping)

'sqrt_15*g_3_0*(-1.62018517460196*sq_x + 1.08012344973464*sq_z + 0.540061724867322*sq_z) + 2.64575131106459*sqrt_15*g_3_1*y*z - g_3_2*(4.8605555238059*sq_x - 6.48074069840786*sq_y + 1.62018517460197*sq_z) - 7.93725393319377*g_3_3*x*y - 3.24037034920393*g_3_4*x*z - 2.64575131106459*sqrt_15*g_3_5*x*y - sqrt_15*g_3_6*z*(1.08012344973464*x + 2.16024689946929*x)'

In [20]:
export_to_latex(third_order_cartesian["y"])

2.64575131106459 \sqrt{15} \nabla Y^{1}_{3} x z + 12.9614813968157 \nabla Y^{2}_{3} x y - \nabla Y^{3}_{3} \cdot \left(3.96862696659689 x^2 - 7.93725393319377 y^2 + 3.96862696659689 z^2\right) + 12.9614813968157 \nabla Y^{4}_{3} y z - 1.3228756555323 \sqrt{15} \nabla Y^{5}_{3} \left(x^2 - z^2\right)


In [21]:
replace_terms_for_implementation(str(third_order_cartesian["y"]), mapping)

'2.64575131106459*sqrt_15*g_3_1*x*z + 12.9614813968157*g_3_2*x*y - g_3_3*(3.96862696659689*sq_x - 7.93725393319377*sq_y + 3.96862696659689*sq_z) + 12.9614813968157*g_3_4*y*z - 1.3228756555323*sqrt_15*g_3_5*(sq_x - sq_z)'

In [22]:
export_to_latex(third_order_cartesian["z"])

\sqrt{15} \nabla Y^{0}_{3} x \left(1.08012344973464 z + 2.16024689946929 z\right) + 2.64575131106459 \sqrt{15} \nabla Y^{1}_{3} x y - 3.24037034920393 \nabla Y^{2}_{3} x z - 7.93725393319377 \nabla Y^{3}_{3} y z - \nabla Y^{4}_{3} \cdot \left(1.62018517460197 x^2 - 6.48074069840786 y^2 + 4.8605555238059 z^2\right) + 2.64575131106459 \sqrt{15} \nabla Y^{5}_{3} y z - \sqrt{15} \nabla Y^{6}_{3} \cdot \left(1.08012344973464 x^{2} + 0.540061724867322 x^2 - 1.62018517460196 z^2\right)


In [23]:
replace_terms_for_implementation(str(third_order_cartesian["z"]), mapping)

'sqrt_15*g_3_0*x*(1.08012344973464*z + 2.16024689946929*z) + 2.64575131106459*sqrt_15*g_3_1*x*y - 3.24037034920393*g_3_2*x*z - 7.93725393319377*g_3_3*y*z - g_3_4*(1.62018517460197*sq_x - 6.48074069840786*sq_y + 4.8605555238059*sq_z) + 2.64575131106459*sqrt_15*g_3_5*y*z - sqrt_15*g_3_6*(1.08012344973464*sq_x + 0.540061724867322*sq_x - 1.62018517460196*sq_z)'

## Fourth order spherical harmonics

In [24]:
y_4_0 = (3 / 4) * math.sqrt(2) * (y_3_0 * z + y_3_6 * x)
y_4_1 = (
    (3 / 4) * y_3_0 * y
    + (3 / 8) * math.sqrt(6) * y_3_1 * z
    + (3 / 8) * math.sqrt(6) * y_3_5 * x
)
y_4_2 = (
    -3 / 56 * math.sqrt(14) * y_3_0 * z
    + (3 / 14) * math.sqrt(21) * y_3_1 * y
    + (3 / 56) * math.sqrt(210) * y_3_2 * z
    + (3 / 56) * math.sqrt(210) * y_3_4 * x
    + (3 / 56) * math.sqrt(14) * y_3_6 * x
)
y_4_3 = (
    -3 / 56 * math.sqrt(42) * y_3_1 * z
    + (3 / 28) * math.sqrt(105) * y_3_2 * y
    + (3 / 28) * math.sqrt(70) * y_3_3 * x
    + (3 / 56) * math.sqrt(42) * y_3_5 * x
)
y_4_4 = (
    -3 / 28 * math.sqrt(42) * y_3_2 * x
    + (3 / 7) * math.sqrt(7) * y_3_3 * y
    - 3 / 28 * math.sqrt(42) * y_3_4 * z
)
y_4_5 = (
    -3 / 56 * math.sqrt(42) * y_3_1 * x
    + (3 / 28) * math.sqrt(70) * y_3_3 * z
    + (3 / 28) * math.sqrt(105) * y_3_4 * y
    - 3 / 56 * math.sqrt(42) * y_3_5 * z
)
y_4_6 = (
    -3 / 56 * math.sqrt(14) * y_3_0 * x
    - 3 / 56 * math.sqrt(210) * y_3_2 * x
    + (3 / 56) * math.sqrt(210) * y_3_4 * z
    + (3 / 14) * math.sqrt(21) * y_3_5 * y
    - 3 / 56 * math.sqrt(14) * y_3_6 * z
)
y_4_7 = (
    -3 / 8 * math.sqrt(6) * y_3_1 * x
    + (3 / 8) * math.sqrt(6) * y_3_5 * z
    + (3 / 4) * y_3_6 * y
)
y_4_8 = (3 / 4) * math.sqrt(2) * (-y_3_0 * x + y_3_6 * z)

In [25]:
fourth_order = {}
for index, expr in enumerate(
    [y_4_0, y_4_1, y_4_2, y_4_3, y_4_4, y_4_5, y_4_6, y_4_7, y_4_8]
):
    fourth_order[str(index)] = take_derivative(expr, [x, y, z])

In [26]:
fourth_order_cartesian = collect_derivatives(fourth_order, 4)

In [27]:
replace_terms_for_implementation(str(fourth_order_cartesian["x"]), mapping)

'-sqrt_15*g_4_0*(3.43693177121688*sq_x*z + 3.43693177121688*sq_x*z - 1.14564392373896*cu_z - 1.14564392373896*cu_z) + sqrt_15*g_4_1*y*(-4.8605555238059*sq_x + 3.24037034920393*sq_z + 1.62018517460197*sq_z) - g_4_2*(0.649519052838329*sqrt_15*sq_x*z - 2.77555756156289e-17*sqrt_15*sq_x*z + 7.54672942406179*sq_x*z - 2.59807621135332*sqrt_15*sq_y*z - 10.0623058987491*sq_y*z + 0.21650635094611*sqrt_15*cu_z + 2.51557647468726*cu_z) - g_4_3*y*(0.918558653543692*sqrt_15*sq_x + 16.0090306546024*sq_x - 9.48683298050514*sq_y + 0.918558653543692*sqrt_15*sq_z + 5.33634355153414*sq_z + 0.459279326771846*sqrt_15*(sq_x - sq_z)) + g_4_4*(-9.0*x*sq_y + 2.25*x*sq_z - 9.0*x*sq_y + 2.25*x*sq_z + 4.5*cu_x) - g_4_5*y*z*(-0.918558653543692*sqrt_15*x + 10.6726871030683*x + 1.83711730708738*sqrt_15*x) - g_4_6*(2.59807621135332*sqrt_15*x*sq_y - 0.21650635094611*sqrt_15*x*sq_z + 2.51557647468726*x*sq_z + 10.0623058987491*x*sq_y - 2.51557647468726*x*sq_z + 0.21650635094611*sqrt_15*x*sq_z - 5.03115294937453*cu_x - 0

In [28]:
export_to_latex(fourth_order_cartesian["x"])

- \sqrt{15} \nabla Y^{0}_{4} \cdot \left(3.43693177121688 x^{2} z + 3.43693177121688 x^2 z - 1.14564392373896 z^{3} - 1.14564392373896 z^3\right) + \sqrt{15} \nabla Y^{1}_{4} y \left(- 4.8605555238059 x^2 + 3.24037034920393 z^{2} + 1.62018517460197 z^2\right) - \nabla Y^{2}_{4} \cdot \left(0.649519052838329 \sqrt{15} x^{2} z - 2.77555756156289 \cdot 10^{-17} \sqrt{15} x^2 z + 7.54672942406179 x^2 z - 2.59807621135332 \sqrt{15} y^{2} z - 10.0623058987491 y^2 z + 0.21650635094611 \sqrt{15} z^{3} + 2.51557647468726 z^3\right) - \nabla Y^{3}_{4} y \left(0.918558653543692 \sqrt{15} x^2 + 16.0090306546024 x^2 - 9.48683298050514 y^2 + 0.918558653543692 \sqrt{15} z^{2} + 5.33634355153414 z^2 + 0.459279326771846 \sqrt{15} \left(x^2 - z^2\right)\right) + \nabla Y^{4}_{4} \left(- 9.0 x y^{2} + 2.25 x z^{2} - 9.0 x y^2 + 2.25 x z^2 + 4.5 x^3\right) - \nabla Y^{5}_{4} y z \left(- 0.918558653543692 \sqrt{15} x + 10.6726871030683 x + 1.83711730708738 \sqrt{15} x\right) - \nabla Y^{6}_{4} \cdot \left(

In [29]:
replace_terms_for_implementation(str(fourth_order_cartesian["y"]), mapping)

'sqrt_15*g_4_1*x*(-1.62018517460197*sq_x + 3.24037034920393*sq_z + 1.62018517460197*sq_z) + g_4_2*x*z*(5.19615242270663*sqrt_15*y + 20.1246117974981*y) - g_4_3*x*(5.33634355153414*sq_x - 28.4604989415154*sq_y + 0.918558653543692*sqrt_15*sq_z + 5.33634355153414*sq_z + 0.459279326771846*sqrt_15*(sq_x - sq_z)) - g_4_4*(9.0*sq_x*y + 9.0*sq_x*y + 9.0*y*sq_z + 9.0*y*sq_z - 12.0*cu_y) - g_4_5*z*(0.918558653543692*sqrt_15*sq_x + 5.33634355153414*sq_x - 28.4604989415154*sq_y + 5.33634355153414*sq_z - 0.459279326771846*sqrt_15*(sq_x - sq_z)) - g_4_6*(10.0623058987491*sq_x*y - 10.0623058987491*y*sq_z + 2.59807621135332*sqrt_15*y*(sq_x - sq_z)) - sqrt_15*g_4_7*z*(3.24037034920393*sq_x + 1.62018517460197*sq_x - 1.62018517460197*sq_z)'

In [30]:
export_to_latex(fourth_order_cartesian["y"])

\sqrt{15} \nabla Y^{1}_{4} x \left(- 1.62018517460197 x^2 + 3.24037034920393 z^{2} + 1.62018517460197 z^2\right) + \nabla Y^{2}_{4} x z \left(5.19615242270663 \sqrt{15} y + 20.1246117974981 y\right) - \nabla Y^{3}_{4} x \left(5.33634355153414 x^2 - 28.4604989415154 y^2 + 0.918558653543692 \sqrt{15} z^{2} + 5.33634355153414 z^2 + 0.459279326771846 \sqrt{15} \left(x^2 - z^2\right)\right) - \nabla Y^{4}_{4} \cdot \left(9.0 x^{2} y + 9.0 x^2 y + 9.0 y z^{2} + 9.0 y z^2 - 12.0 y^3\right) - \nabla Y^{5}_{4} z \left(0.918558653543692 \sqrt{15} x^{2} + 5.33634355153414 x^2 - 28.4604989415154 y^2 + 5.33634355153414 z^2 - 0.459279326771846 \sqrt{15} \left(x^2 - z^2\right)\right) - \nabla Y^{6}_{4} \cdot \left(10.0623058987491 x^{2} y - 10.0623058987491 y z^{2} + 2.59807621135332 \sqrt{15} y \left(x^2 - z^2\right)\right) - \sqrt{15} \nabla Y^{7}_{4} z \left(3.24037034920393 x^{2} + 1.62018517460197 x^2 - 1.62018517460197 z^2\right)


In [31]:
replace_terms_for_implementation(str(fourth_order_cartesian["z"]), mapping)

'-sqrt_15*g_4_0*(1.14564392373896*cu_x - 3.43693177121688*x*sq_z - 3.43693177121688*x*sq_z + 1.14564392373896*cu_x) + sqrt_15*g_4_1*x*y*(3.24037034920393*z + 6.48074069840786*z) - g_4_2*(0.21650635094611*sqrt_15*cu_x - 2.59807621135332*sqrt_15*x*sq_y - 10.0623058987491*x*sq_y + 0.649519052838329*sqrt_15*x*sq_z - 2.77555756156289e-17*sqrt_15*x*sq_z + 7.54672942406179*x*sq_z + 2.51557647468726*cu_x) - g_4_3*x*y*(-0.918558653543692*sqrt_15*z + 10.6726871030683*z + 1.83711730708738*sqrt_15*z) + g_4_4*(2.25*sq_x*z + 2.25*sq_x*z - 9.0*sq_y*z - 9.0*sq_y*z + 4.5*cu_z) - g_4_5*y*(0.918558653543692*sqrt_15*sq_x + 5.33634355153414*sq_x - 9.48683298050514*sq_y + 0.918558653543692*sqrt_15*sq_z + 16.0090306546024*sq_z - 0.459279326771846*sqrt_15*(sq_x - sq_z)) + g_4_6*(-0.21650635094611*sqrt_15*sq_x*z + 2.51557647468726*sq_x*z - 2.51557647468726*sq_x*z + 0.21650635094611*sqrt_15*sq_x*z + 2.59807621135332*sqrt_15*sq_y*z + 10.0623058987491*sq_y*z - 5.03115294937453*cu_z - 0.433012701892219*sqrt_15*cu_

In [32]:
export_to_latex(fourth_order_cartesian["z"])

- \sqrt{15} \nabla Y^{0}_{4} \cdot \left(1.14564392373896 x^{3} - 3.43693177121688 x z^{2} - 3.43693177121688 x z^2 + 1.14564392373896 x^3\right) + \sqrt{15} \nabla Y^{1}_{4} x y \left(3.24037034920393 z + 6.48074069840786 z\right) - \nabla Y^{2}_{4} \cdot \left(0.21650635094611 \sqrt{15} x^{3} - 2.59807621135332 \sqrt{15} x y^{2} - 10.0623058987491 x y^2 + 0.649519052838329 \sqrt{15} x z^{2} - 2.77555756156289 \cdot 10^{-17} \sqrt{15} x z^2 + 7.54672942406179 x z^2 + 2.51557647468726 x^3\right) - \nabla Y^{3}_{4} x y \left(- 0.918558653543692 \sqrt{15} z + 10.6726871030683 z + 1.83711730708738 \sqrt{15} z\right) + \nabla Y^{4}_{4} \cdot \left(2.25 x^{2} z + 2.25 x^2 z - 9.0 y^{2} z - 9.0 y^2 z + 4.5 z^3\right) - \nabla Y^{5}_{4} y \left(0.918558653543692 \sqrt{15} x^{2} + 5.33634355153414 x^2 - 9.48683298050514 y^2 + 0.918558653543692 \sqrt{15} z^2 + 16.0090306546024 z^2 - 0.459279326771846 \sqrt{15} \left(x^2 - z^2\right)\right) + \nabla Y^{6}_{4} \left(- 0.21650635094611 \sqrt{15} x

## Fifth order spherical harmonics

These have yet to be implemented in EquiTriton, but goes to show that higher order derivatives can be easily obtained.

In [33]:
y_5_0 = (1 / 10) * math.sqrt(110) * (y_4_0 * z + y_4_8 * x)
y_5_1 = (
    (1 / 5) * math.sqrt(11) * y_4_0 * y
    + (1 / 5) * math.sqrt(22) * y_4_1 * z
    + (1 / 5) * math.sqrt(22) * y_4_7 * x
)
y_5_2 = (
    -1 / 30 * math.sqrt(22) * y_4_0 * z
    + (4 / 15) * math.sqrt(11) * y_4_1 * y
    + (1 / 15) * math.sqrt(154) * y_4_2 * z
    + (1 / 15) * math.sqrt(154) * y_4_6 * x
    + (1 / 30) * math.sqrt(22) * y_4_8 * x
)
y_5_3 = (
    -1 / 30 * math.sqrt(66) * y_4_1 * z
    + (1 / 15) * math.sqrt(231) * y_4_2 * y
    + (1 / 30) * math.sqrt(462) * y_4_3 * z
    + (1 / 30) * math.sqrt(462) * y_4_5 * x
    + (1 / 30) * math.sqrt(66) * y_4_7 * x
)
y_5_4 = (
    -1 / 15 * math.sqrt(33) * y_4_2 * z
    + (2 / 15) * math.sqrt(66) * y_4_3 * y
    + (1 / 15) * math.sqrt(165) * y_4_4 * x
    + (1 / 15) * math.sqrt(33) * y_4_6 * x
)
y_5_5 = (
    -1 / 15 * math.sqrt(110) * y_4_3 * x
    + (1 / 3) * math.sqrt(11) * y_4_4 * y
    - 1 / 15 * math.sqrt(110) * y_4_5 * z
)
y_5_6 = (
    -1 / 15 * math.sqrt(33) * y_4_2 * x
    + (1 / 15) * math.sqrt(165) * y_4_4 * z
    + (2 / 15) * math.sqrt(66) * y_4_5 * y
    - 1 / 15 * math.sqrt(33) * y_4_6 * z
)
y_5_7 = (
    -1 / 30 * math.sqrt(66) * y_4_1 * x
    - 1 / 30 * math.sqrt(462) * y_4_3 * x
    + (1 / 30) * math.sqrt(462) * y_4_5 * z
    + (1 / 15) * math.sqrt(231) * y_4_6 * y
    - 1 / 30 * math.sqrt(66) * y_4_7 * z
)
y_5_8 = (
    -1 / 30 * math.sqrt(22) * y_4_0 * x
    - 1 / 15 * math.sqrt(154) * y_4_2 * x
    + (1 / 15) * math.sqrt(154) * y_4_6 * z
    + (4 / 15) * math.sqrt(11) * y_4_7 * y
    - 1 / 30 * math.sqrt(22) * y_4_8 * z
)
y_5_9 = (
    -1 / 5 * math.sqrt(22) * y_4_1 * x
    + (1 / 5) * math.sqrt(22) * y_4_7 * z
    + (1 / 5) * math.sqrt(11) * y_4_8 * y
)
y_5_10 = (1 / 10) * math.sqrt(110) * (-y_4_0 * x + y_4_8 * z)

In [34]:
fifth_order = {}
for index, expr in enumerate(
    [y_5_0, y_5_1, y_5_2, y_5_3, y_5_4, y_5_5, y_5_6, y_5_7, y_5_8, y_5_9, y_5_10]
):
    fifth_order[str(index)] = take_derivative(expr, [x, y, z])

In [35]:
fifth_order_cartesian = collect_derivatives(fifth_order, 5)

In [36]:
replace_terms_for_implementation(str(fifth_order_cartesian["x"]), mapping)

'-sqrt_15*dY5^0*(10.8140533566281*sq_x*sq_z + 1.80234222610469*sq_x*sq_z + 5.40702667831406*sq_x*sq_z - 3.00390371017448*sq_x * sq_x - 1.20156148406979*sq_z * sq_z - 1.80234222610469*sq_z * sq_z) - sqrt_15*dY5^1*y*(11.399013115178*sq_x*z + 11.399013115178*sq_x*z - 3.79967103839267*cu_z - 3.79967103839267*cu_z) - sqrt_15*dY5^10*(1.20156148406979*x*cu_z - 4.80624593627917*cu_x*z + 7.20936890441875*x*cu_z + 3.60468445220938*x*cu_z - 7.20936890441875*cu_x*z) - dY5^2*(12.4869932329604*sq_x*sq_y + 1.07470926301023*sqrt_15*sq_x*sq_z - 3.12174830824011*sq_x*sq_z + 0.537354631505117*sqrt_15*sq_x*sq_z + 7.52296484107164*sqrt_15*sq_x*sq_y - 0.537354631505117*sqrt_15*sq_x*sq_z + 9.36524492472033*sq_x*sq_z - 5.20291384706685*sq_x * sq_x - 0.895591052508528*sqrt_15*sq_x * sq_x - 5.01530989404776*sqrt_15*sq_y*sq_z - 2.50765494702388*sqrt_15*sq_y*sq_z - 12.4869932329604*sq_y*sq_z + 0.358236421003411*sqrt_15*sq_z * sq_z + 0.179118210501706*sqrt_15*sq_z * sq_z + 3.12174830824011*sq_z * sq_z) - dY5^3*(5.

In [37]:
replace_terms_for_implementation(str(fifth_order_cartesian["y"]), mapping)

'-3.79967103839267*sqrt_15*dY5^1*(cu_x*z - x*cu_z - x*cu_z + cu_x*z) + dY5^2*(-8.32466215530697*cu_x*y + 24.9739864659209*x*y*sq_z + 10.0306197880955*sqrt_15*x*y*sq_z + 5.01530989404776*sqrt_15*x*y*sq_z - 5.01530989404776*sqrt_15*cu_x*y) - dY5^3*(1.75499287747842*sqrt_15*cu_x*z - 7.89746794865291*sqrt_15*x*sq_y*z - 71.369110965459*x*sq_y*z + 1.75499287747842*sqrt_15*x*cu_z + 10.1955872807799*x*cu_z + 10.1955872807799*cu_x*z) - dY5^4*(11.5607093207986*cu_x*y + 11.5607093207986*x*y*sq_z + 3.97994974842648*sqrt_15*x*y*sq_z - 1.98997487421324*sqrt_15*x*y*sq_z + 19.2678488679977*x*y*sq_z - 51.3809303146605*x*cu_y + 1.98997487421324*sqrt_15*cu_x*y + 19.2678488679977*cu_x*y) + dY5^5*(-34.8245602987317*sq_x*sq_y + 1.28452325786651*sqrt_15*sq_x*sq_z - 0.321130814466628*sqrt_15*sq_x*sq_z + 4.9749371855331*sq_x*sq_z - 14.9248115565993*sq_x*sq_y - 0.321130814466628*sqrt_15*sq_x*sq_z + 4.9749371855331*sq_x*sq_z + 0.321130814466628*sqrt_15*sq_x * sq_x + 4.9749371855331*sq_x * sq_x - 14.9248115565993

In [38]:
replace_terms_for_implementation(str(fifth_order_cartesian["z"]), mapping)

'-sqrt_15*dY5^0*(1.20156148406979*cu_x*z + 7.20936890441875*cu_x*z - 4.80624593627917*x*cu_z - 7.20936890441875*x*cu_z + 3.60468445220938*cu_x*z) - sqrt_15*dY5^1*y*(3.79967103839267*cu_x - 11.399013115178*x*sq_z - 11.399013115178*x*sq_z + 3.79967103839267*cu_x) + sqrt_15*dY5^10*(1.20156148406979*sq_x * sq_x - 10.8140533566281*sq_x*sq_z - 5.40702667831406*sq_x*sq_z - 1.80234222610469*sq_x*sq_z + 1.80234222610469*sq_x * sq_x + 3.00390371017448*sq_z * sq_z) - dY5^2*(-2.08116553882674*cu_x*z + 0.358236421003411*sqrt_15*cu_x*z + 0.716472842006822*sqrt_15*cu_x*z - 5.01530989404776*sqrt_15*x*sq_y*z - 10.0306197880955*sqrt_15*x*sq_y*z - 24.9739864659209*x*sq_y*z + 1.43294568401365*sqrt_15*x*cu_z + 0.716472842006823*sqrt_15*x*cu_z + 12.4869932329604*x*cu_z - 0.358236421003411*sqrt_15*cu_x*z + 6.24349661648022*cu_x*z) - dY5^3*(1.75499287747842*sqrt_15*cu_x*y - 2.63248931621764*sqrt_15*x*cu_y + 5.26497863243527*sqrt_15*x*y*sq_z + 2.77555756156289e-17*sqrt_15*x*y*sq_z + 30.5867618423396*x*y*sq_z -